# Generate the OCR synthdata

1. Take dataset map that just extracts the articles.
2. Split each one by word, with meta-data that keeps track of order of sentences so it can be pieced back together.
3. Create new dataset, and also function to map back to old dataset post inference.
4. Check it all works.

In [1]:
# load the dataset
from datasets import load_dataset

dataset = load_dataset(
    "multi_eurlex",
    "all_languages",
    label_level="level_3",
    trust_remote_code=True,
)

dataset

DatasetDict({
    train: Dataset({
        features: ['celex_id', 'text', 'labels'],
        num_rows: 55000
    })
    test: Dataset({
        features: ['celex_id', 'text', 'labels'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['celex_id', 'text', 'labels'],
        num_rows: 5000
    })
})

In [4]:
from arc_spice.data.multieurlex_utils import load_multieurlex

dataset = load_multieurlex(
    data_dir="../data", level=3, lang_pair={"source": "fr", "target": "en"}
)

In [6]:
t_dataset = dataset[0][0]
t_dataset

Dataset({
    features: ['celex_id', 'labels', 'source_text', 'target_text', 'class_labels'],
    num_rows: 55000
})

In [25]:
# lets look at an example
eg = next(iter(t_dataset))

print(f"labels: \n {eg['labels']}")
print(f"celex id: \n {eg['celex_id']}")
print(f"text: \n {eg['source_text'].split(' ')}")
print(f"class labels: \n {eg['class_labels']}")

labels: 
 [57, 565, 129, 30, 45, 212]
celex id: 
 32006D0213
text: 
 ['\nArticle', 'premier\nLes', 'produits', 'et/ou', 'matériaux', 'de', 'construction', 'qui', 'satisfont', 'à', 'l’ensemble', 'des', 'prescriptions', 'relatives', 'à', 'la', 'caractéristique', '«réaction', 'au', 'feu»', 'sans', 'devoir', 'subir', 'd’essais', 'complémentaires', 'sont', 'énumérés', 'à', 'l’annexe.\nArticle', '2\nLes', 'classes', 'spécifiques', 'à', 'appliquer', 'aux', 'différents', 'produits', 'et/ou', 'matériaux', 'de', 'construction', 'au', 'sein', 'de', 'la', 'classification', 'des', 'caractéristiques', 'de', 'réaction', 'au', 'feu', 'adoptées', 'par', 'la', 'décision', '2000/147/CE', 'sont', 'indiquées', 'à', 'l’annexe', 'de', 'la', 'présente', 'décision.\nArticle', '3\nLes', 'produits', 'sont', 'considérés', 'au', 'regard', 'des', 'conditions', 'de', 'leur', 'utilisation', 'finale,', 'le', 'cas', 'échéant.\nArticle', '4\nLes', 'États', 'membres', 'sont', 'destinataires', 'de', 'la', 'présente', 'déc

Ok, so we want to generate OCR data here, with a source image and a target text, that is split by word in such a way that it can be easily pieced back together into the original text snippets.

The OCR generator method 'from Dict' just gives pre generated stuff, this isn't useful. Will have to maintain the dictionary structure myself.

In [24]:
from trdg.generators import GeneratorFromStrings


def make_ocr_data(example):
    """Take an input data row and transform it into OCR version word by word"""
    text_split = example["source_text"].split(" ")
    generator = GeneratorFromStrings(text_split, count=len(text_split))
    return [
        {
            "image": gen[0],
            "label": gen[1],
            "word_idx": idx,
            # "orig_idx": example_idx,
            "orig_data": {
                "celex_id": example["celex_id"],
                "class_labels": example["class_labels"],
                "target_text": example["target_text"],
            },
        }
        for idx, gen in enumerate(generator)
    ]

In [38]:
new_data = t_dataset.map(make_ocr_data, kwarg)

Map:   0%|          | 0/55000 [00:00<?, ? examples/s]

TypeError: make_ocr_data() missing 1 required positional argument: 'example_idx'

In [35]:
new_data = []

count = 0

for idx, data_point in enumerate(t_dataset):
    transformed_data = make_ocr_data(data_point, idx)
    new_data += transformed_data
    if transformed_data[0]["orig_idx"] == 2:
        break

In [37]:
import datasets
import pandas as pd

new_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=new_data))

ArrowInvalid: ('Could not convert <PIL.Image.Image image mode=RGB size=84x32 at 0x2CFD36D70> with type Image: did not recognize Python value type when inferring an Arrow data type', 'Conversion failed for column image with type object')